In [9]:
import pandas as pd
import json
import mysql.connector
import csv

In [66]:
csv_file = "data/Crimes_2001_to_Present.csv"
df = pd.read_csv(csv_file)

In [67]:
df.shape

(7846809, 22)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7846809 entries, 0 to 7846808
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  int64  
 11  District              float64
 12  Ward                  float64
 13  Community Area        float64
 14  FBI Code              object 
 15  X Coordinate          float64
 16  Y Coordinate          float64
 17  Year                  int64  
 18  Updated On            object 
 19  Latitude              float64
 20  Longitude             float64
 21  Location              object 
dtypes: bool(2), float64(7), int64(3), object(1

In [69]:
df.isnull().sum()

ID                           0
Case Number                  4
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description     10758
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614849
Community Area          613476
FBI Code                     0
X Coordinate             88111
Y Coordinate             88111
Year                         0
Updated On                   0
Latitude                 88111
Longitude                88111
Location                 88111
dtype: int64

## Reapplying transformations

#### Creating Time column

In [70]:
# turning date column into datetime to extract the time and create the time column
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
df['Time'] = df['Date'].apply(lambda x: x.time())
move_time = df.pop('Time')
df.insert(3,'Time', move_time)

##### Changing Updated On date format

In [71]:
# changing updated_on format
df['Updated On'] = pd.to_datetime(df['Updated On'], format='%m/%d/%Y %I:%M:%S %p')

#### Adjussting Date column as date only

In [72]:
# taking just the date part, converting into dtype object
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p').dt.date

#### Replacing null values

In [73]:
# replacing nulls
df = df.fillna({'Case Number': 'NULL', 'Location Description': 'NULL'})
df = df.fillna({'District': 0, 'Ward': 0, 'Community Area': 0})

In [74]:
cols = ['District', 'Ward', 'Community Area', 'Arrest', 'Domestic']
df[cols] = df[cols].astype(int)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7846809 entries, 0 to 7846808
Data columns (total 23 columns):
 #   Column                Dtype         
---  ------                -----         
 0   ID                    int64         
 1   Case Number           object        
 2   Date                  object        
 3   Time                  object        
 4   Block                 object        
 5   IUCR                  object        
 6   Primary Type          object        
 7   Description           object        
 8   Location Description  object        
 9   Arrest                int32         
 10  Domestic              int32         
 11  Beat                  int64         
 12  District              int32         
 13  Ward                  int32         
 14  Community Area        int32         
 15  FBI Code              object        
 16  X Coordinate          float64       
 17  Y Coordinate          float64       
 18  Year                  int64         
 19  

In [76]:
df.head()

,ID,Case Number,Date,Time,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11646166,JC213529,2018-09-01,00:01:00,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,0,...,8,44,06,NaN,NaN,2018,2019-04-06 16:04:43,NaN,NaN,NaN
1,11645836,JC212333,2016-05-01,00:25:00,055XX S ROCKWELL ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NULL,0,...,15,63,11,NaN,NaN,2016,2019-04-06 16:04:43,NaN,NaN,NaN
2,11449702,JB373031,2018-07-31,13:30:00,009XX E HYDE PARK BLVD,2024,NARCOTICS,POSS: HEROIN(WHITE),STREET,1,...,5,41,18,NaN,NaN,2018,2019-04-09 16:24:58,NaN,NaN,NaN
3,11643334,JC209972,2018-12-19,16:30:00,056XX W WELLINGTON AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,0,...,31,19,14,NaN,NaN,2018,2019-04-04 16:16:11,NaN,NaN,NaN
4,11645527,JC212744,2015-02-02,10:00:00,069XX W ARCHER AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,0,...,23,56,11,NaN,NaN,2015,2019-04-06 16:04:43,NaN,NaN,NaN


In [77]:
df.columns = ["id","case_number","date", "time", "block","iucr","primary_type","description","location_desc","arrest","domestic","beat","district","ward", "community_area", "fbi_code", "x_coord",
       "y_coord","year","updated_on","latitude","longitude", "location"]
df.columns

Index(['id', 'case_number', 'date', 'time', 'block', 'iucr', 'primary_type',
       'description', 'location_desc', 'arrest', 'domestic', 'beat',
       'district', 'ward', 'community_area', 'fbi_code', 'x_coord', 'y_coord',
       'year', 'updated_on', 'latitude', 'longitude', 'location'],
      dtype='object')

In [78]:
not_null_mask = ~df['latitude'].isnull() & ~df['longitude'].isnull()

# Apply the transformation only for non-null values
df.loc[not_null_mask, 'location'] = df[not_null_mask].apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

print(df)

               id case_number        date      time                   block  \
0        11646166    JC213529  2018-09-01  00:01:00   082XX S INGLESIDE AVE   
1        11645836    JC212333  2016-05-01  00:25:00     055XX S ROCKWELL ST   
2        11449702    JB373031  2018-07-31  13:30:00  009XX E HYDE PARK BLVD   
3        11643334    JC209972  2018-12-19  16:30:00  056XX W WELLINGTON AVE   
4        11645527    JC212744  2015-02-02  10:00:00      069XX W ARCHER AVE   
...           ...         ...         ...       ...                     ...   
7846804  13128007    JG325985  2023-06-21  20:00:00  031XX N CALIFORNIA AVE   
7846805  13129172    JG327619  2023-06-20  04:00:00   028XX N MAPLEWOOD AVE   
7846806  13128066    JG325838  2023-06-06  15:42:00    018XX N LOCKWOOD AVE   
7846807  13128324    JG326502  2023-05-13  12:00:00       020XX W CERMAK RD   
7846808  13128375    JG326564  2023-06-24  13:29:00    069XX N HAMILTON AVE   

         iucr                primary_type  \
0     

In [79]:
df= df.dropna(subset=['location'])

In [80]:
df.shape

(7758698, 23)

In [81]:
df.isnull().sum()

id                0
case_number       0
date              0
time              0
block             0
iucr              0
primary_type      0
description       0
location_desc     0
arrest            0
domestic          0
beat              0
district          0
ward              0
community_area    0
fbi_code          0
x_coord           0
y_coord           0
year              0
updated_on        0
latitude          0
longitude         0
location          0
dtype: int64

In [82]:
df.to_csv('data/data.csv', index=False)

#### Taking half of the data to upload

In [86]:
sampled_df = df.sample(frac=0.5, random_state=42)

In [88]:
sampled_df.shape

(3879349, 23)

#### Loading the data into the database

In [87]:
sampled_df.to_csv('data/data_sample.csv', index=False)

In [83]:
with open('config_db.json') as config_json:
    config = json.load(config_json)

conx = mysql.connector.connect(**config) 

In [91]:
mycursor = conx.cursor()

mycursor.execute("""CREATE TABLE IF NOT EXISTS crimes (
                 id int PRIMARY KEY,
                 case_number VARCHAR(10),
                 date date, 
                 time time, 
                 block VARCHAR(150), 
                 iucr VARCHAR(10), 
                 primary_type VARCHAR(150), 
                 description VARCHAR(150), 
                 location_desc VARCHAR(150), 
                 arrest boolean, 
                 domestic boolean, 
                 beat int, 
                 district int, 
                 ward int, 
                 community_area int, 
                 fbi_code VARCHAR(5), 
                 x_coord VARCHAR(15), 
                 y_coord VARCHAR(15), 
                 year int, 
                 updated_on datetime, 
                 latitude float, 
                 longitude float, 
                 location point)""")

query = "DESCRIBE crimes"
mycursor.execute(query)

description_table = mycursor.fetchall()

mycursor.close()

# we have seen the creation of the table 
desc_table=pd.DataFrame(description_table, columns=['Field', 'Type', 'Null', 'Key', 'Default', 'Extra'])
print(desc_table)

mycursor.close()

             Field             Type Null  Key Default Extra
0               id           b'int'   NO  PRI    None      
1      case_number   b'varchar(10)'  YES         None      
2             date          b'date'  YES         None      
3             time          b'time'  YES         None      
4            block  b'varchar(150)'  YES         None      
5             iucr   b'varchar(10)'  YES         None      
6     primary_type  b'varchar(150)'  YES         None      
7      description  b'varchar(150)'  YES         None      
8    location_desc  b'varchar(150)'  YES         None      
9           arrest    b'tinyint(1)'  YES         None      
10        domestic    b'tinyint(1)'  YES         None      
11            beat           b'int'  YES         None      
12        district           b'int'  YES         None      
13            ward           b'int'  YES         None      
14  community_area           b'int'  YES         None      
15        fbi_code    b'varchar(5)'  YES

False

In [90]:
#Delete table, just in case :)... delete because of varchar's lengths
# mycursor = conx.cursor()

# query = "DROP table crimes"
# mycursor.execute(query)

# mycursor.close()

True

In [92]:
mycursor = conx.cursor()

query2 = """INSERT INTO crimes (id,case_number,date, time, block, iucr, primary_type, description, location_desc, arrest, domestic, beat, district, ward, community_area, fbi_code, x_coord, y_coord, year, updated_on, latitude, longitude, location) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,ST_GeomFromText(%s))"""


with open('data/data_sample.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader) # ignore column names
        for row in reader:
            datos= (row[0], row[1], row[2], row[3],
                                  row[4], row[5], row[6], row[7],
                                  row[8], row[9], row[10], row[11],
                                  row[12], row[13], row[14], row[15], row[16], row[17],
                                  row[18], row[19], row[20], row[21], row[22])
            mycursor.execute(query2, datos)
    
conx.commit()

mycursor.close()

True

In [94]:
mycursor = conx.cursor()

query3 = """ALTER TABLE crimes
DROP case_number, DROP domestic, DROP beat, DROP ward, DROP community_area, DROP fbi_code, DROP x_coord, DROP y_coord, DROP latitude, DROP longitude"""

mycursor.execute(query3)
conx.commit()

mycursor.close()

True

#### We have to delete the rows of iucr column in crimes that do not appear in codes, beacuse these are not in the official IUCR dataset.

In [ ]:
mycursor = conx.cursor()

add_fk = """DELETE FROM crimes
            WHERE iucr NOT IN (SELECT iucr FROM codes);"""

mycursor.execute(add_fk)
conx.commit()
mycursor.close()

In [96]:
mycursor = conx.cursor()

add_fk = """ALTER TABLE crimes
            ADD CONSTRAINT fk_iucr_codes
            FOREIGN KEY (iucr)
            REFERENCES codes(iucr)"""

mycursor.execute(add_fk)
conx.commit()
mycursor.close()

True

In [97]:
mycursor = conx.cursor()

query = "DESCRIBE crimes"
mycursor.execute(query)

description_table = mycursor.fetchall()

mycursor.close()

# we have seen the creation of the table 
desc_table=pd.DataFrame(description_table, columns=['Field', 'Type', 'Null', 'Key', 'Default', 'Extra'])
print(desc_table)

            Field             Type Null  Key Default Extra
0              id           b'int'   NO  PRI    None      
1            date          b'date'  YES         None      
2            time          b'time'  YES         None      
3           block  b'varchar(150)'  YES         None      
4            iucr   b'varchar(10)'  YES  MUL    None      
5   location_desc  b'varchar(150)'  YES         None      
6          arrest    b'tinyint(1)'  YES         None      
7        district           b'int'  YES         None      
8            year           b'int'  YES         None      
9      updated_on      b'datetime'  YES         None      
10       location         b'point'  YES         None      
